In [1]:
import ollama
import os
import json
import numpy as np
from numpy.linalg import norm
import PyPDF2
from docx import Document
import streamlit as st

In [2]:
# Membaca file TXT
def read_txt(filename):
    with open(filename, "r", encoding="utf-8-sig") as f:
        return f.read()

# Membaca file PDF
def read_pdf(filename):
    with open(filename, "rb") as f:
        pdf_reader = PyPDF2.PdfReader(f)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

# Membaca file DOCX
def read_docx(filename):
    doc = Document(filename)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    return text

# Mendapatkan paragraf dari semua file
def parse_file(filename):
    if filename.endswith(".txt"):
        content = read_txt(filename)
    elif filename.endswith(".pdf"):
        content = read_pdf(filename)
    elif filename.endswith(".docx"):
        content = read_docx(filename)
    else:
        raise ValueError(f"Unsupported file type: {filename}")
    
    paragraphs = []
    buffer = []
    for line in content.splitlines():
        line = line.strip()
        if line:
            buffer.append(line)
        elif len(buffer):
            paragraphs.append(" ".join(buffer))
            buffer = []
    if len(buffer):
        paragraphs.append(" ".join(buffer))
    return paragraphs

In [3]:
# Mendapatkan paragraf dari semua file
def parse_file(filename):
    if filename.endswith(".txt"):
        content = read_txt(filename)
    elif filename.endswith(".pdf"):
        content = read_pdf(filename)
    elif filename.endswith(".docx"):
        content = read_docx(filename)
    else:
        raise ValueError(f"Unsupported file type: {filename}")
    
    paragraphs = []
    buffer = []
    for line in content.splitlines():
        line = line.strip()
        if line:
            buffer.append(line)
        elif len(buffer):
            paragraphs.append(" ".join(buffer))
            buffer = []
    if len(buffer):
        paragraphs.append(" ".join(buffer))
    return paragraphs

# Simpan embedding ke file
def save_embeddings(filename, embeddings):
    if not os.path.exists("embeddings"):
        os.makedirs("embeddings")
    with open(f"embeddings/{filename}.json", "w") as f:
        json.dump(embeddings, f)

# Memuat embedding dari file
def load_embeddings(filename):
    if not os.path.exists(f"embeddings/{filename}.json"):
        return False
    with open(f"embeddings/{filename}.json", "r") as f:
        return json.load(f)

# Mendapatkan embedding
def get_embeddings(filename, modelname, chunks):
    if (embeddings := load_embeddings(filename)) is not False:
        return embeddings
    embeddings = [
        ollama.embeddings(model=modelname, prompt=chunk)["embedding"]
        for chunk in chunks
    ]
    save_embeddings(filename, embeddings)
    return embeddings

# Cosine similarity untuk menemukan kemiripan
def find_most_similar(needle, haystack):
    needle_norm = norm(needle)
    similarity_scores = [
        np.dot(needle, item) / (needle_norm * norm(item)) for item in haystack
    ]
    return sorted(zip(similarity_scores, range(len(haystack))), reverse=True)

In [ ]:
import requests

try:
    response = requests.post("http://localhost:11434/api/chat", json={
                    "model": "llama3",
                    "messages": [
                        {
                            "role": "system",
                            "content": "nama saya adalah teguh prasetyo umur 25 tahun, pekerjaan dba" ,
                        },
                        {"role": "user", "content": "siapa teguh?"},
                    ]
                })
    print(response.json())
except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

In [4]:
import requests
import json

try:
    # response = requests.post("http://localhost:11434/api/generate", json={"model": "llama3", "prompt": "what is AI?"})
    
    print("Response Status Code:", response.status_code)
    print("Response Content-Type:", response.headers.get('Content-Type'))

    if response.headers.get('Content-Type') == 'application/x-ndjson':
        print("Response is in NDJSON format.")
        
        full_response = ""  # Initialize an empty string to hold the full response
        
        for line in response.text.splitlines():
            try:
                # Parse each line as a JSON object
                json_data = json.loads(line)
                
                # Append the 'response' part of each JSON object to the full response
                full_response += json_data['response']
                
                # If the response is complete, print the full response
                if json_data.get('done'):
                    print("Full Response:", full_response)
                    break
                
            except json.JSONDecodeError as e:
                print(f"Error parsing line: {line}")
                print(f"JSONDecodeError: {e}")
    else:
        print("Response is not in NDJSON format.")
        print(response.json())

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


NameError: name 'response' is not defined

In [34]:
import os
import requests

SYSTEM_PROMPT = """Anda adalah asisten yang membantu menjawab pertanyaan dengan bahasa Indonesia 
dan berdasarkan cuplikan teks yang diberikan dalam konteks. Jawab hanya menggunakan konteks yang disediakan, 
menjadi sesingkat mungkin. Jika Anda tidak yakin, katakan saja Anda tidak tahu.
Context:
"""

data_folder = "data"
all_paragraphs = []
filenames = []

# Function to parse files (You need to implement this or use an existing function)
def parse_file(file_path):
    # Parse the file and return paragraphs (implement your file parsing logic here)
    return []

# Function to get embeddings (using requests instead of ollama)
def get_embeddings(modelname, chunks):
    embeddings = []
    for chunk in chunks:
        try:
            response = requests.post(
                "http://localhost:11434/api/embeddings",  # Use the correct endpoint here
                json={"model": modelname, "prompt": chunk}
            )
            response.raise_for_status()  # Ensure we raise an error for bad status codes
            embeddings.append(response.json()["embedding"])
        except requests.exceptions.RequestException as e:
            print(f"Error fetching embeddings: {e}")
            embeddings.append(None)  # Append None or handle this error gracefully
    return embeddings

# Function to find the most similar chunks (implement your similarity search here)
def find_most_similar(prompt_embedding, embeddings):
    # Implement your similarity function here
    return []

def main():
    global all_paragraphs, filenames

    # Iterasi semua file dalam folder data
    for file in os.listdir(data_folder):
        file_path = os.path.join(data_folder, file)
        if file.lower().endswith((".txt", ".pdf", ".docx")):
            paragraphs = parse_file(file_path)
            all_paragraphs.extend(paragraphs)
            filenames.append(file)

    # Create embeddings manually using requests
    embeddings = get_embeddings("nomic-embed-text", all_paragraphs)

    while True:
        prompt = input("Silakan tanya bosku? (ketik 'exit' untuk keluar) -> ")

        if prompt.lower() == "exit":
            print("Exiting the assistant. Goodbye!")
            break

        try:
            # Manually fetch the prompt embedding using requests
            prompt_response = requests.post(
                "http://localhost:11434/api/embeddings",  # Use the correct endpoint here
                json={"model": "nomic-embed-text", "prompt": prompt}
            )
            prompt_response.raise_for_status()  # Ensure we raise an error for bad status codes
            prompt_embedding = prompt_response.json()["embedding"]
            
            most_similar_chunks = find_most_similar(prompt_embedding, embeddings)[:5]

            # Get the chat response manually
            chat_response = requests.post(
                "http://localhost:11434/api/chat",  # Use the correct endpoint here
                json={
                    "model": "llama3",
                    "messages": [
                        {
                            "role": "system",
                            "content": SYSTEM_PROMPT + "\n".join(all_paragraphs[item[1]] for item in most_similar_chunks),
                        },
                        {"role": "user", "content": prompt},
                    ]
                }
            )
            if chat_response.status_code != 200:
                print(f"Error: Received {chat_response.status_code} response")
                print(f"Response content: {chat_response.text}")
            else:
                print("\n\n")
                print(chat_response.json()["message"]["content"])

        except requests.exceptions.RequestException as e:
            print(f"Error during request: {e}")

if __name__ == "__main__":
    main()


Silakan tanya bosku? (ketik 'exit' untuk keluar) ->  hai bot





Error during request: Extra data: line 2 column 1 (char 120)


Silakan tanya bosku? (ketik 'exit' untuk keluar) ->  exit


Exiting the assistant. Goodbye!


In [49]:
import os
import json
import time
import requests
import PyPDF2
from docx import Document
import numpy as np
from numpy.linalg import norm

# URL untuk API Ollama, ganti sesuai dengan alamat API Anda
OLLAMA_API_URL = "http://localhost:11434/api/"

# Fungsi untuk request embeddings
def get_embeddings_from_ollama(modelname, chunk):
    response = requests.post(
        OLLAMA_API_URL + "embeddings",
        json={"model": modelname, "prompt": chunk},
    )
    response.raise_for_status()
    return response.json()["embedding"]

# Fungsi untuk request chat (LLM) response
import requests
import json

# Fungsi untuk request chat (LLM) response
def chat_with_ollama(model, messages):
    try:
        response = requests.post(
            OLLAMA_API_URL + "chat",
            json={"model": model, "messages": messages},
        )
        
        if response.headers.get('Content-Type') == 'application/x-ndjson':
            # print("Response is in NDJSON format.")
            
            full_response = ""  # Initialize an empty string to hold the full response
            
            for line in response.text.splitlines():
                try:
                    # Parse each line as a JSON object
                    json_data = json.loads(line)
                    
                    # Print the entire json_data to inspect its structure
                    # print("Parsed JSON Data:", json_data)
                    
                    # Check if 'message' key exists and contains 'content'
                    if 'message' in json_data and 'content' in json_data['message']:
                        # Append the 'content' part of each 'message' to the full response
                        full_response += json_data['message']['content']
                        
                        # If the response is complete, break out of the loop
                        if json_data.get('done'):
                            break
                    else:
                        print("No 'message' or 'content' key in this JSON object:", json_data)
                
                except json.JSONDecodeError as e:
                    print(f"Error parsing line: {line}")
                    print(f"JSONDecodeError: {e}")
        
            # Return the full response after the loop ends
            return full_response

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")
        return None



# Membaca file TXT
def read_txt(filename):
    with open(filename, "r", encoding="utf-8-sig") as f:
        return f.read()

# Membaca file PDF
def read_pdf(filename):
    with open(filename, "rb") as f:
        pdf_reader = PyPDF2.PdfReader(f)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text

# Membaca file DOCX
def read_docx(filename):
    doc = Document(filename)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    return text

# Mendapatkan paragraf dari semua file
def parse_file(filename):
    if filename.endswith(".txt"):
        content = read_txt(filename)
    elif filename.endswith(".pdf"):
        content = read_pdf(filename)
    elif filename.endswith(".docx"):
        content = read_docx(filename)
    else:
        raise ValueError(f"Unsupported file type: {filename}")
    
    paragraphs = []
    buffer = []
    for line in content.splitlines():
        line = line.strip()
        if line:
            buffer.append(line)
        elif len(buffer):
            paragraphs.append(" ".join(buffer))
            buffer = []
    if len(buffer):
        paragraphs.append(" ".join(buffer))
    return paragraphs

# Mendapatkan embedding
def get_embeddings(filename, modelname, chunks):
    embedding_file_path = f"embeddings/{filename}.json"
    
    # Check if the embedding file already exists
    if os.path.exists(embedding_file_path):
        print(f"Embedding file {filename}.json already exists. Loading embeddings...")
        # Load the embeddings from the existing file
        with open(embedding_file_path, "r") as f:
            embeddings = json.load(f)
    else:
        print(f"Embedding file {filename}.json not found. Generating new embeddings...")
        # If file doesn't exist, generate embeddings
        embeddings = [
            get_embeddings_from_ollama(modelname, chunk) for chunk in chunks
        ]
        # Save the embeddings to file
        save_embeddings(filename, embeddings)
    
    return embeddings

# Simpan embedding ke file
def save_embeddings(filename, embeddings):
    if not os.path.exists("embeddings"):
        os.makedirs("embeddings")
    with open(f"embeddings/{filename}.json", "w") as f:
        json.dump(embeddings, f)


# # Mendapatkan embedding
# def get_embeddings(filename, modelname, chunks):
#     embeddings = [
#         get_embeddings_from_ollama(modelname, chunk) for chunk in chunks
#     ]
#     save_embeddings(filename, embeddings)
#     return embeddings

# # Simpan embedding ke file
# def save_embeddings(filename, embeddings):
#     if not os.path.exists("embeddings"):
#         os.makedirs("embeddings")
#     with open(f"embeddings/{filename}.json", "w") as f:
#         json.dump(embeddings, f)

# Cosine similarity untuk menemukan kemiripan
def find_most_similar(needle, haystack):
    needle_norm = norm(needle)
    similarity_scores = [
        np.dot(needle, item) / (needle_norm * norm(item)) for item in haystack
    ]
    return sorted(zip(similarity_scores, range(len(haystack))), reverse=True)


In [53]:
# Fungsi utama
def main():
    SYSTEM_PROMPT = """Anda adalah asisten yang membantu menjawab pertanyaan dengan bahasa Indonesia 
    dan berdasarkan cuplikan teks yang diberikan dalam konteks. Jawab hanya menggunakan konteks yang disediakan, 
    menjadi sesingkat mungkin. Jika Anda tidak yakin, katakan saja Anda tidak tahu.
    Context:
    """

    data_folder = "data"
    all_paragraphs = []
    filenames = []

    # Iterasi semua file dalam folder data
    for file in os.listdir(data_folder):
        file_path = os.path.join(data_folder, file)
        if file.lower().endswith((".txt", ".pdf", ".docx")):
            paragraphs = parse_file(file_path)
            all_paragraphs.extend(paragraphs)
            filenames.append(file)

    # Buat embedding
    embeddings = get_embeddings("data_api", "nomic-embed-text", all_paragraphs)
    
    while True:
        prompt = input("Silakan tanya bosku? (ketik 'exit' untuk keluar) -> ")
        
        if prompt.lower() == "exit":
            print("Exiting the assistant. Goodbye!")
            break

        # Mendapatkan embedding prompt dari Ollama
        prompt_embedding = get_embeddings_from_ollama("nomic-embed-text", prompt)

        # Temukan paragraf yang paling mirip
        most_similar_chunks = find_most_similar(prompt_embedding, embeddings)[:5]

        # Persiapkan konteks untuk chat
        context = "\n".join(all_paragraphs[item[1]] for item in most_similar_chunks)
        
        # Kirim request chat ke Ollama
        response = chat_with_ollama(
            model="llama3",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT + context},
                {"role": "user", "content": prompt},
            ]
        )

        # Print the full response
        if response:
            print("\n\n")
            print(response)  # Output the final model response
        else:
            print("Error or no response received.")

if __name__ == "__main__":
    main()

Embedding file data_api.json already exists. Loading embeddings...


Silakan tanya bosku? (ketik 'exit' untuk keluar) ->  apa itu ojk?





OJK! OJK stands for Otoritas Jasa Keuangan (Financial Services Authority), which is the main financial regulatory body in Indonesia. Its main objective is to maintain the stability and integrity of the financial system, as well as to protect consumers and ensure fair competition among financial institutions.


Silakan tanya bosku? (ketik 'exit' untuk keluar) ->  exit


Exiting the assistant. Goodbye!


In [13]:
SYSTEM_PROMPT = """Anda adalah asisten yang membantu menjawab pertanyaan dengan bahasa Indonesia 
    dan berdasarkan cuplikan teks yang diberikan dalam konteks. Jawab hanya menggunakan konteks yang disediakan, 
    menjadi sesingkat mungkin. Jika Anda tidak yakin, katakan saja Anda tidak tahu.
    Context:
    """

data_folder = "data"
all_paragraphs = []
filenames = []

# Iterasi semua file dalam folder data
for file in os.listdir(data_folder):
    file_path = os.path.join(data_folder, file)
    if file.lower().endswith((".txt", ".pdf", ".docx")):
        paragraphs = parse_file(file_path)
        all_paragraphs.extend(paragraphs)
        filenames.append(file)

# Buat embedding
embeddings = get_embeddings("data_api", "nomic-embed-text", all_paragraphs)

In [39]:

# prompt = input("Silakan tanya bosku? (ketik 'exit' untuk keluar) -> ")

# if prompt.lower() == "exit":
    # print("Exiting the assistant. Goodbye!")
    # break

# Mendapatkan embedding prompt dari Ollama
prompt_embedding = get_embeddings_from_ollama("nomic-embed-text", "apa itu ojk?")

# Temukan paragraf yang paling mirip
most_similar_chunks = find_most_similar(prompt_embedding, embeddings)[:5]

# Persiapkan konteks untuk chat
context = "\n".join(all_paragraphs[item[1]] for item in most_similar_chunks)

# Kirim request chat ke Ollama
response = chat_with_ollama(
    model="llama3",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT + context},
        {"role": "user", "content": "apa itu ojk?"},
    ]
)

# Print the full response
# if response:
    # print("\n\n")
    # print(response)  # Output the final model response
# else:
    # print("Error or no response received.")

Response is in NDJSON format.
Parsed JSON Data: {'model': 'llama3', 'created_at': '2024-12-04T10:15:31.3584388Z', 'message': {'role': 'assistant', 'content': 'Men'}, 'done': False}
Parsed JSON Data: {'model': 'llama3', 'created_at': '2024-12-04T10:15:31.7903087Z', 'message': {'role': 'assistant', 'content': 'ur'}, 'done': False}
Parsed JSON Data: {'model': 'llama3', 'created_at': '2024-12-04T10:15:32.251544Z', 'message': {'role': 'assistant', 'content': 'ut'}, 'done': False}
Parsed JSON Data: {'model': 'llama3', 'created_at': '2024-12-04T10:15:32.813959Z', 'message': {'role': 'assistant', 'content': ' kont'}, 'done': False}
Parsed JSON Data: {'model': 'llama3', 'created_at': '2024-12-04T10:15:33.2786261Z', 'message': {'role': 'assistant', 'content': 'eks'}, 'done': False}
Parsed JSON Data: {'model': 'llama3', 'created_at': '2024-12-04T10:15:33.76007Z', 'message': {'role': 'assistant', 'content': ' yang'}, 'done': False}
Parsed JSON Data: {'model': 'llama3', 'created_at': '2024-12-04T10

In [52]:
try:
    # response = requests.post("http://localhost:11434/api/generate", json={"model": "llama3", "prompt": "what is AI?"})
    
    print("Response Status Code:", response.status_code)
    print("Response Content-Type:", response.headers.get('Content-Type'))

    if response.headers.get('Content-Type') == 'application/x-ndjson':
        print("Response is in NDJSON format.")
        
        full_response = ""  # Initialize an empty string to hold the full response
        
        for line in response.text.splitlines():
            try:
                # Parse each line as a JSON object
                json_data = json.loads(line)
                
                # Append the 'response' part of each JSON object to the full response
                full_response += json_data['response']
                
                # If the response is complete, print the full response
                if json_data.get('done'):
                    print("Full Response:", full_response)
                    break
                
            except json.JSONDecodeError as e:
                print(f"Error parsing line: {line}")
                print(f"JSONDecodeError: {e}")
    else:
        print("Response is not in NDJSON format.")
        print(response.json())

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


AttributeError: 'str' object has no attribute 'status_code'

In [51]:
try:
    # response = requests.post("http://localhost:11434/api/generate", json={"model": "llama3", "prompt": "what is AI?"})
    
    print("Response Status Code:", response.status_code)
    print("Response Content-Type:", response.headers.get('Content-Type'))

    if response.headers.get('Content-Type') == 'application/x-ndjson':
        print("Response is in NDJSON format.")
        
        full_response = ""  # Initialize an empty string to hold the full response
        
        for line in response.text.splitlines():
            try:
                # Parse each line as a JSON object
                json_data = json.loads(line)
                
                # Print the entire json_data to inspect its structure
                print("Parsed JSON Data:", json_data)
                
                # Check if 'message' key exists and contains 'content'
                if 'message' in json_data and 'content' in json_data['message']:
                    # Append the 'content' part of each 'message' to the full response
                    full_response += json_data['message']['content']
                    
                    # If the response is complete, print the full response
                    if json_data.get('done'):
                        print("Full Response:", full_response)
                        break
                else:
                    print("No 'message' or 'content' key in this JSON object:", json_data)
                    
            except json.JSONDecodeError as e:
                print(f"Error parsing line: {line}")
                print(f"JSONDecodeError: {e}")
    else:
        print("Response is not in NDJSON format.")
        print(response.json())

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")

AttributeError: 'str' object has no attribute 'status_code'